<a href="https://colab.research.google.com/github/zhirongwang94/csm148/blob/main/final_project/first_approach.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is the final project. 
Background:

1) id: unique identifier

2) gender: "Male", "Female" or "Other"

3) age: age of the patient

4) hypertension: 0 if the patient doesn't have hypertension, 1 if the patient has hypertension 

5) heart_disease: 0 if the patient doesn't have any heart diseases, 1 if the patient has a heart disease

6) ever_married: "No" or "Yes"

7) work_type: "children", "Govt_jov", "Never_worked", "Private" or "Self-employed"

8) Residence_type: "Rural" or "Urban"

9) avg_glucose_level: average glucose level in blood

10) bmi: body mass index

11) smoking_status: "formerly smoked", "never smoked", "smokes" or "Unknown"*

12) stroke: 1 if the patient had a stroke or 0 if not

##Part 1. Load the Data



In [140]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [141]:
!ls ./gdrive/MyDrive/datasets/stroke

dataset-stroke.csv


In [142]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

csv_path = './gdrive/MyDrive/datasets/stroke/dataset-stroke.csv'
stroke = pd.read_csv(csv_path)

stroke.head()

,Unnamed: 0,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,2764,20292,Female,24.0,0,0,Yes,Private,Urban,85.55,63.3,never smoked,0
1,98,8154,Male,57.0,1,0,Yes,Govt_job,Urban,78.92,27.7,formerly smoked,1
2,3735,27176,Female,69.0,0,0,Yes,Private,Rural,103.73,34.7,never smoked,0
3,4500,13598,Male,60.0,0,0,Yes,Self-employed,Urban,227.23,40.0,formerly smoked,0
4,3254,27034,Female,65.0,0,0,Yes,Govt_job,Urban,82.72,29.8,smokes,0


# Part2 Data Analysis and Imputation

In [143]:
stroke.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2310 entries, 0 to 2309
Data columns (total 13 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   Unnamed: 0         2310 non-null   int64  
 1   id                 2310 non-null   int64  
 2   gender             2310 non-null   object 
 3   age                2310 non-null   float64
 4   hypertension       2310 non-null   int64  
 5   heart_disease      2310 non-null   int64  
 6   ever_married       2310 non-null   object 
 7   work_type          2310 non-null   object 
 8   Residence_type     2310 non-null   object 
 9   avg_glucose_level  2310 non-null   float64
 10  bmi                2215 non-null   float64
 11  smoking_status     2310 non-null   object 
 12  stroke             2310 non-null   int64  
dtypes: float64(3), int64(5), object(5)
memory usage: 234.7+ KB


In [144]:
stroke.describe()

,Unnamed: 0,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2310.000000,2215.000000,2310.000000
mean,2530.838095,35961.380519,43.202043,0.096537,0.054545,105.747004,28.805418,0.052814
std,1483.865940,21191.909528,22.784886,0.295390,0.227140,44.984524,7.800871,0.223710
min,0.000000,77.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,1217.250000,17007.750000,25.250000,0.000000,0.000000,77.007500,23.600000,0.000000
50%,2573.500000,36047.000000,44.000000,0.000000,0.000000,91.850000,28.100000,0.000000
75%,3810.750000,54231.500000,61.000000,0.000000,0.000000,113.452500,32.900000,0.000000
max,5109.000000,72918.000000,82.000000,1.000000,1.000000,271.740000,92.000000,1.000000


In [145]:
# .info() result shows there are some row in bmi has 95 Nah value
# let check if they have there 'Unknown' values in smoking_staus. 
stroke[stroke['smoking_status'] == 'Unknown']

,Unnamed: 0,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
7,3877,45983,Male,21.0,0,0,No,Private,Urban,56.79,20.4,Unknown,0
8,1680,70752,Male,37.0,0,0,Yes,Private,Urban,145.26,26.7,Unknown,0
10,4211,19498,Female,81.0,0,1,No,Self-employed,Urban,99.44,27.7,Unknown,0
14,445,61465,Male,13.0,0,0,No,children,Rural,55.39,23.2,Unknown,0
17,2677,45461,Female,70.0,0,0,Yes,Private,Urban,91.28,30.1,Unknown,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2299,288,70970,Female,17.0,0,0,No,Self-employed,Urban,82.18,23.4,Unknown,0
2300,1610,71721,Female,18.0,0,0,No,Private,Rural,80.06,31.8,Unknown,0
2304,2114,25996,Female,29.0,0,0,Yes,Private,Urban,81.20,23.0,Unknown,0
2305,1812,12366,Female,35.0,0,0,No,Private,Urban,97.58,24.3,Unknown,0


In [146]:
# There are 95 Nah values in bmi column, and there are 705 'Unknown' values in Smoking_status. 
# In the first appraoch, I try to remove all the rows with Nah values and the Smoking_status column. 

# remove rows with Nah values. 
row_has_Nah = stroke.isnull().any(axis=1)
incomplete_rows = stroke[row_has_Nah]
stroke = stroke.drop(incomplete_rows.index, axis=0)
# stroke.info()

# remove smoking_status column. 
stroke = stroke.drop(['id','smoking_status'], axis=1)

# Part3 Data preprocess. 

In [150]:
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import category_encoders as ce

encoder= ce.OrdinalEncoder(return_df=True, mapping=[{'col':'gender','mapping':{'Male':0,'Female':1}}])


# from sklearn.base import BaseEstimator, TransformerMixin

categorical_features = ['work_type', 'gender', 'ever_married', 'Residence_type']
numerical_features = ['age', 'hypertension', 'heart_disease', 'avg_glucose_level', 'bmi']


full_pipeline = ColumnTransformer([
        ("num", StandardScaler(), numerical_features ),                          
        # ("cat", OneHotEncoder(),  categorical_features),
        ("cat", OneHotEncoder(), categorical_features),
    ])


stroke_prepared = full_pipeline.fit_transform(stroke)
stroke_target = stroke['stroke']


In [177]:
from sklearn.model_selection import train_test_split
data_target = stroke['stroke']
X_train, X_test, y_train, y_test = train_test_split(stroke_prepared, stroke_target, test_size=0.3, random_state=0)

y.mean()

0.03935483870967742

# Part4  Basic Logistic Regression

In [176]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression

clf = LogisticRegression(random_state=0)
clf.fit(X_train, y_train)
clf.score(X_test, y_test)


0.9428571428571428

# Part5 PCA

In [182]:
import numpy as np
from sklearn.decomposition import PCA
# X = np.array([[-1, -1], [-2, -1], [-3, -2], [1, 1], [2, 1], [3, 2]])
pca = PCA(n_components=10)
# pca.fit(X)
# PCA(n_components=2)
# print(pca.explained_variance_ratio_)
# [0.9924... 0.0075...]
# print(pca.singular_values_)
# [6.30061... 0.54980...]
pca.fit(X_train)

pca.explained_variance_ratio_

array([0.28568383, 0.14326885, 0.12428125, 0.11896593, 0.09430252,
       0.07117956, 0.06554841, 0.04208968, 0.0253015 , 0.01929674])

# Part6 Neural Network

In [186]:
from sklearn.neural_network import MLPClassifier
from sklearn.datasets import make_classification


# >>> X, y = make_classification(n_samples=100, random_state=1)
# >>> X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y,
# ...                                                     random_state=1)
clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

clf.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/neural_network/_multilayer_perceptron.py:571: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (300) reached and the optimization hasn't converged yet.
  % self.max_iter, ConvergenceWarning)


0.9383458646616541

# Part7 Cross Validation

In [212]:
import numpy as np
from sklearn.model_selection import KFold


# >>> kf.get_n_splits(X)
# 2
# print(kf)
kf = KFold(n_splits=2, random_state=None, shuffle=False)
# clf = MLPClassifier(random_state=1, max_iter=300).fit(X_train, y_train)

X = stroke_prepared;
y = stroke['stroke'].to_numpy()
for train_index, test_index in kf.split(X):
  X_train, X_test = X[train_index], X[test_index]
  y_train, y_test = y[train_index], y[test_index]
  clf = MLPClassifier(random_state=1, max_iter=1000).fit(X_train, y_train)
  print(clf.score(X_test, y_test))

0.9377256317689531
0.9367660343270099


In [198]:
stroke_prepared.shape

(2215, 16)